In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
df_raw = pd.read_csv('/Users/adityasingh/Downloads/Bank-Marketing-Dataset/train.csv', delimiter=';', encoding='UTF-8')
df_raw.head()
df_raw.shape

(45211, 17)

In [3]:
df_raw.describe(include = 'all')

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
count,45211.000000,45211,45211,45211,45211,45211.000000,45211,45211,45211,45211.000000,45211,45211.000000,45211.000000,45211.000000,45211.000000,45211,45211
unique,NaN,12,3,4,2,NaN,2,2,3,NaN,12,NaN,NaN,NaN,NaN,4,2
top,NaN,blue-collar,married,secondary,no,NaN,yes,no,cellular,NaN,may,NaN,NaN,NaN,NaN,unknown,no
freq,NaN,9732,27214,23202,44396,NaN,25130,37967,29285,NaN,13766,NaN,NaN,NaN,NaN,36959,39922
mean,40.936210,NaN,NaN,NaN,NaN,1362.272058,NaN,NaN,NaN,15.806419,NaN,258.163080,2.763841,40.197828,0.580323,NaN,NaN
std,10.618762,NaN,NaN,NaN,NaN,3044.765829,NaN,NaN,NaN,8.322476,NaN,257.527812,3.098021,100.128746,2.303441,NaN,NaN
min,18.000000,NaN,NaN,NaN,NaN,-8019.000000,NaN,NaN,NaN,1.000000,NaN,0.000000,1.000000,-1.000000,0.000000,NaN,NaN
25%,33.000000,NaN,NaN,NaN,NaN,72.000000,NaN,NaN,NaN,8.000000,NaN,103.000000,1.000000,-1.000000,0.000000,NaN,NaN
50%,39.000000,NaN,NaN,NaN,NaN,448.000000,NaN,NaN,NaN,16.000000,NaN,180.000000,2.000000,-1.000000,0.000000,NaN,NaN
75%,48.000000,NaN,NaN,NaN,NaN,1428.000000,NaN,NaN,NaN,21.000000,NaN,319.000000,3.000000,-1.000000,0.000000,NaN,NaN


Results from EDA above:
Numeric Data:
1) Age range from 18 - 95, mean = 40, std = 10.6
2) Balance - 1362 mean, std = 3044
Binary Data:
1) Default history - Yes / No
2) Housing loan - Yes / No
3) Personal Loan - Yes / No
Categorical Data:
1) Job - 12 categories
2) Marital - 3 categories
3) Education - 4 categories

9) contact: contact communication type (categorical: "unknown","telephone","cellular")
10) day: last contact day of the month (numeric)
11) month: last contact month of year (categorical: "jan", "feb", "mar", …, "nov", "dec")
12) duration: last contact duration, in seconds (numeric)
13) campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
14) pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
15) previous: number of contacts performed before this campaign and for this client (numeric)
16) poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

In [4]:
df_raw.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

Binary variables to be mapped to 0 & 1 respectively

1) Default
2) Housing
3) Loan

In [5]:
df_raw['default'].replace({'no':0, 'yes':1},inplace=True)
df_raw['housing'].replace({'yes': 1, 'no': 0}, inplace=True)
df_raw['loan'].replace({'no':0, 'yes':1},inplace=True)

**Copying data frame to a new data frame before appying getdummies method for one-hot encoding**

In [6]:
df_work = df_raw.copy()
df_work.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,no


*Note that month column has <u>not</u> been converted to dummy values*

In [7]:
df_dummy = pd.get_dummies(df_work, columns=['job','marital','education','contact','poutcome'], drop_first=True)

In [8]:
scaler = StandardScaler()

*Copying data to a new df again <u>after applying one-hot encoding</u> and before applying standardization*

In [9]:
# Applying standardization transformations to selected variables
df_standardized = df_dummy.copy()
df_standardized['age'] = scaler.fit_transform(df_standardized[['age']])
df_standardized['balance'] = scaler.fit_transform(df_standardized[['balance']])
df_standardized['duration'] = scaler.fit_transform(df_standardized[['duration']])
df_standardized['campaign'] = scaler.fit_transform(df_standardized[['campaign']])
df_standardized['pdays'] = scaler.fit_transform(df_standardized[['pdays']])
df_standardized['previous'] = scaler.fit_transform(df_standardized[['previous']])
df_standardized.head()

,age,default,balance,housing,loan,day,month,duration,campaign,pdays,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,1.606965,0,0.256419,1,0,5,may,0.011016,-0.569351,-0.411453,...,True,False,False,True,False,False,True,False,False,True
1,0.288529,0,-0.437895,1,0,5,may,-0.416127,-0.569351,-0.411453,...,False,True,True,False,False,False,True,False,False,True
2,-0.747384,0,-0.446762,1,1,5,may,-0.707361,-0.569351,-0.411453,...,True,False,True,False,False,False,True,False,False,True
3,0.571051,0,0.047205,1,0,5,may,-0.645231,-0.569351,-0.411453,...,True,False,False,False,True,False,True,False,False,True
4,-0.747384,0,-0.447091,0,0,5,may,-0.233620,-0.569351,-0.411453,...,False,True,False,False,True,False,True,False,False,True


In [10]:
# Renaming columns for easier comprehension
df_standardized.rename(columns={'campaign':'number of contacts'}, inplace=True)
df_standardized.rename(columns={'previous': 'previous # contacts'}, inplace=True)
df_standardized.rename(columns={'pdays':'#days gap b/w contact'}, inplace=True)

In [11]:
## Replacing the '-1' which meant not contacted to '0'
df_standardized['#days gap b/w contact'].replace({-1: 0}, inplace=True)

Initiate and populate the train variables i.e. x_train and y_train

In [12]:
columns_order = ['age',
 'default',
 'balance',
 'housing',
 'loan',
 'day',
 'month',
 'duration',
 'number of contacts',
 '#days gap b/w contact',
 'previous # contacts',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_unknown',
 'marital_married',
 'marital_single',
 'education_secondary',
 'education_tertiary',
 'education_unknown',
 'contact_telephone',
 'contact_unknown',
 'poutcome_other',
 'poutcome_success',
 'poutcome_unknown',
'y']

In [13]:
df_standardized = df_standardized[columns_order]

In [14]:
x_train = df_standardized.iloc[:,0:-1]

In [15]:
x_train.drop('month', inplace=True, axis=1)
x_train.shape

(45211, 31)

In [16]:
y_train = df_standardized.iloc[:,-1]

In [17]:
y_train.replace({'no':0, 'yes':1}, inplace=True)

In [18]:
df_test = pd.read_csv('/Users/adityasingh/Downloads/Bank-Marketing-Dataset/test.csv', encoding='UTF-8', delimiter=';')
df_test.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [19]:
df_test.shape

(4521, 17)

In [20]:
df_test = pd.get_dummies(df_test, columns=['job','marital','education','contact','poutcome'], drop_first=True)

In [21]:
df_test['age'] = scaler.fit_transform(df_test[['age']])
df_test['balance'] = scaler.fit_transform(df_test[['balance']])
df_test['duration'] = scaler.fit_transform(df_test[['duration']])
df_test['campaign'] = scaler.fit_transform(df_test[['campaign']])
df_test['pdays'] = scaler.fit_transform(df_test[['pdays']])
df_test['previous'] = scaler.fit_transform(df_test[['previous']])

In [22]:
df_test.rename(columns={'campaign':'number of contacts'}, inplace=True)
df_test.rename(columns={'pdays': '#days gap b/w contact'}, inplace=True)
df_test.rename(columns={'previous': 'previous # contacts'}, inplace=True)

In [23]:
df_test['#days gap b/w contact'].replace({-1: 0}, inplace=True)

In [24]:
df_test.head()
df_test.shape

(4521, 33)

In [25]:
df_test = df_test[columns_order]

In [26]:
x_test = df_test.iloc[:,0:-1]

In [27]:
x_test.drop('month', inplace=True, axis =1)

In [28]:
x_test.head()

,age,default,balance,housing,loan,day,duration,number of contacts,#days gap b/w contact,previous # contacts,...,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,contact_telephone,contact_unknown,poutcome_other,poutcome_success,poutcome_unknown
0,-1.056270,no,0.121072,no,no,19,-0.711861,-0.576829,-0.407218,-0.320413,...,True,False,False,False,False,False,False,False,False,True
1,-0.772583,no,1.118644,yes,yes,11,-0.169194,-0.576829,2.989044,2.041734,...,True,False,True,False,False,False,False,False,False,False
2,-0.583458,no,-0.024144,yes,no,16,-0.303898,-0.576829,2.899143,0.270124,...,False,True,False,True,False,False,False,False,False,False
3,-1.056270,no,0.017726,yes,yes,3,-0.250017,0.387967,-0.407218,-0.320413,...,True,False,False,True,False,False,True,False,False,True
4,1.686036,no,-0.472753,yes,no,5,-0.146102,-0.576829,-0.407218,-0.320413,...,True,False,True,False,False,False,True,False,False,True


In [37]:
x_test.shape

(4521, 31)

In [29]:
y_test = df_test.iloc[:,-1]

In [30]:
y_test.replace({'no':0, 'yes':1}, inplace=True)

In [31]:
y_train.shape

(45211,)

# Logistic Regression

*Using GridSearchCV to tune hyperparameters for Logistic Regression*

In [32]:
param_grid_LR = {
    'solver' : ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    'C' : [0.1, 1, 10, 100]
}

In [33]:
# LR = LogisticRegression(max_iter=1000)
# clf = GridSearchCV(LR, param_grid_LR)
# clf.fit(x_train, y_train)

In [34]:
# clf.best_params_

In [35]:
# Best parameters for Logistic Regression: C : 1, solver : 'liblinear' as per above test
LR = LogisticRegression(random_state = 0, max_iter = 1000, C=1, solver='liblinear').fit(x_train, y_train)

In [36]:
yhat_lr = LR.predict(x_test)

ValueError: could not convert string to float: 'no'

Checking Model Accuracy For Logistic Regression

Confusion Matrix

In [ ]:
import seaborn as sns
cm_lr = confusion_matrix(y_test, yhat_lr)

In [ ]:
sns.heatmap(cm_lr/np.sum(cm_lr), annot=True, fmt='.2%', cmap='Blues')

In [ ]:
print (classification_report(y_test, yhat_lr))

# K Nearest Neighbours

In [ ]:
# Ks = 10
# mean_acc = np.zeros((Ks))

# for n in range(1,Ks):
    
#     #Train Model and Predict  
#     neigh = KNeighborsClassifier(n_neighbors = n).fit(x_train.values,y_train.values)
#     yhat=neigh.predict(x_test.values)
#     mean_acc[n] = accuracy_score(y_test, yhat)

# mean_acc

We will select k = 3 as this gives us the highest accuracy and again print classification report

In [ ]:
k = 3
Knn = KNeighborsClassifier(n_neighbors=k)
Knn.fit(x_train.values, y_train.values)
yhat_knn = Knn.predict(x_test.values)

In [ ]:
cm_knn = confusion_matrix(y_test, yhat_knn)
sns.heatmap(cm_knn/np.sum(cm_knn), annot=True, cmap='Blues', fmt='.2%')

In [ ]:
print (classification_report(y_test, yhat_knn))

We find that KNearestNeighbors is better at classifying the given dataset basis classification report accuracy

# Support Vector Machine

In [ ]:
SVM_df = svm.SVC()

In [ ]:
# ,'poly','sigmoid'
parameters_grid_SVC = {'kernel':('linear', 'rbf','poly','sigmoid'),
              'C' : [0.1, 1, 10],
              }

In [ ]:
# clf = GridSearchCV(SVM_df, parameters_grid_SVC, n_jobs=-1)
# clf.fit(x_train, y_train)

In [ ]:
# clf.best_params_

In [ ]:
# {'C': 0.1, 'kernel': 'linear'} as per clf.best_params_, GridSearchCV takes approx 40 minutes to fit x_train and y_train data
clf_svm = svm.SVC(kernel='linear', C=0.1,probability=True)
clf_svm.fit(x_train, y_train)

In [ ]:
yhat_svm = clf_svm.predict(x_test)

*Checking accuracy*

In [ ]:
# Confusion matrix
cm_svm = confusion_matrix(y_test, yhat_svm)
sns.heatmap(cm_svm / np.sum(cm_svm), cmap = 'Blues', annot = True, fmt='.2%')

In [ ]:
print(classification_report(y_test, yhat_svm))

# Naive Bayes

*Gaussian distribution*

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB().fit(x_train, y_train)
yhat = clf.predict(x_test)
print(classification_report(y_test, yhat))

*Bernoulli Distribution with binarize set to True*

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf_nb = BernoulliNB(binarize=True).fit(x_train, y_train)
yhat_nb = clf_nb.predict(x_test)
print(classification_report(y_test, yhat_nb))

In [ ]:
cm_nb = confusion_matrix(y_test,yhat)
sns.heatmap(cm_nb/np.sum(cm_nb), fmt = '.2%', cmap='Blues', annot=True)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf_dt = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=5, min_samples_split=20, min_samples_leaf=10, random_state=0)
clf_dt.fit(x_train, y_train)
yhat_dt = clf_dt.predict(x_test)
print(classification_report(y_test, yhat_dt))

In [ ]:
cm_dt = confusion_matrix(y_test, yhat_dt)
sns.heatmap(cm_dt/np.sum(cm_dt), annot=True, fmt = '.2%', cmap='Blues')

*Cost Complexity Pruning*

In [ ]:
from sklearn import tree
from matplotlib import pyplot as plt
plt.figure(figsize=(20,20))
tree.plot_tree(clf_dt,filled = True)

In [ ]:
path = clf_dt.cost_complexity_pruning_path(x_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
ccp_alphas

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(x_train, y_train)
    clfs.append(clf)

*Accuracy vs Alpha for training & testing sets*

In [ ]:
print (x_train.shape, x_test.shape)

In [ ]:
train_scores = [clf.score(x_train, y_train) for clf in clfs]
test_scores = [clf.score(x_test, y_test) for clf in clfs]
fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")   
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",drawstyle="steps-post")
ax.legend()
plt.show()

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=50, criterion='gini', n_jobs= -1, random_state=42, max_depth=5, min_samples_split=20, min_samples_leaf=10)
clf_rf.fit(x_train, y_train)
yhat_rf = clf.predict(x_test)
cm_rf = confusion_matrix(y_test, yhat_rf)
sns.heatmap(cm_rf/np.sum(cm_rf), annot=True, fmt='0.2%', cmap = 'Blues')

In [ ]:
print(classification_report(y_test, yhat_rf))

# Selecting the best classification algorithm

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

*Calculating ROC curve for Logistic Regression*

In [ ]:
yhat_lr_proba = LR.predict_proba(x_test)[:,1]
print(yhat_lr_proba)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, yhat_lr_proba)
AUC_LR = roc_auc_score(y_test, yhat_lr_proba)
plt.plot(fpr, tpr, linewidth = 4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Logistic Regression')
plt.show()

In [ ]:
print (AUC_LR*100)

*Calculating ROC curve for SVM*

In [ ]:
yhat_svm_proba = clf_svm.predict_proba(x_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, yhat_svm_proba)
AUC_SVM = roc_auc_score(y_test, yhat_svm_proba)
plt.plot(fpr, tpr, linewidth = 4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for KNN')
plt.show()

In [ ]:
print(AUC_SVM*100)

*Calculating ROC for Naive Bayes*

In [ ]:
yhat_nb_proba = clf_nb.predict_proba(x_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, yhat_nb_proba)
AUC_NB = roc_auc_score(y_test, yhat_nb_proba)
plt.plot(fpr, tpr, linewidth = 4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Naive Bayes')
plt.show()
print (AUC_NB*100)

*Calculating ROC for Decision Tree*

In [ ]:
yhat_dt_proba = clf_dt.predict_proba(x_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, yhat_dt_proba)
AUC_DT = roc_auc_score(y_test, yhat_dt_proba)
plt.plot(fpr, tpr, linewidth = 4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Decision Tree')
plt.show()
print (AUC_DT*100)

*Calculating ROC for Random Forest*

In [ ]:
yhat_rf_proba = clf_rf.predict_proba(x_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, yhat_rf_proba)
AUC_RF = roc_auc_score(y_test, yhat_rf_proba)
plt.plot(fpr, tpr, linewidth = 4)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Random Forest')
plt.show()
print (AUC_RF*100)

Selected KNN basis best accuracy + greatest area under ROC